In [5]:
!pip install pysolr pyeuropeana python-dotenv

In [13]:
import requests
import json
import os

from pysolr import Solr

In [9]:
%load_ext dotenv
%dotenv

In [21]:
SOLR_ENDPOINT = 'http://localhost:8984/solr/pangia'
EUROPEANA_ENDPOINT = 'https://www.europeana.eu/api/v2/search.json'

params = {
    'query': 'climate',
    'rows': 100,
    'wskey': os.getenv("EUROPEANA_API_KEY")
}

solr = Solr(SOLR_ENDPOINT)

In [33]:
# https://github.com/europeana/rd-europeana-python-api#usage
import pyeuropeana.apis as apis
import pyeuropeana.utils as utils

results = apis.search(
    query = '*',
    # qf = '(skos_concept:"http://data.europeana.eu/concept/base/48" AND TYPE:IMAGE)',
    reusability = 'open AND permission',
    media = True,
    thumbnail = True,
    landingpage = True,
    colourpalette = '#0000FF',
    theme = 'photography',
    sort = 'europeana_id',
    profile = 'rich',
    rows = 10,
) # this gives you full response metadata along with cultural heritage object metadata

# print(results)

# use this utility function to transform a subset of the cultural heritage object metadata
# into a readable Pandas DataFrame
# dataframe = utils.search2df(results)
# dataframe

{'apikey': 'rvKVUnBrg', 'success': True, 'requestNumber': 999, 'itemsCount': 10, 'totalResults': 218, 'nextCursor': 'AoE3LzkxNjEyMy9TTE1faXRlbV80MTEyNTk=', 'items': [{'completeness': 7, 'country': ['France'], 'dataProvider': ['National Library of France'], 'dcCreator': ['Cande, Daniel (1938-....). Photographe'], 'dcCreatorLangAware': {'def': ['Cande, Daniel (1938-....). Photographe']}, 'dcDescription': ['Appartient à l’ensemble documentaire : PhoSpec1'], 'dcDescriptionLangAware': {'def': ['Appartient à l’ensemble documentaire : PhoSpec1']}, 'dcLanguage': ['fra', 'fra'], 'dcLanguageLangAware': {'def': ['fra', 'fra']}, 'dcSubjectLangAware': {'fre': ['Opéra national de Paris. Ballet -- Photographies']}, 'dcTitleLangAware': {'def': ['[Recueil. Photographies. Hommage à Alwin Nikolaïs. Paris. Opéra Garnier. 1994 / Daniel Cande]']}, 'dcTypeLangAware': {'def': ['http://data.europeana.eu/concept/48'], 'fre': ['image fixe', 'photographie'], 'eng': ['image', 'still image', 'photograph']}, 'edmCon

In [41]:
for item in results['items']:
    doc = {
            'id': item['id'],
            'title': item['title'],
            'description': item.get('description'),
            'thumbnail': item['edmPreview'],
            'record_url': item['link'],
            'licence': item['rights'],
            'provider': item['dataProvider']
        }
    
    solr.add(doc)
    
solr.commit()

solr.optimize()

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n</response>\n'

In [44]:
results = solr.search("provider:*")
print("\nSaw {0} result(s).".format(len(results)))

for result in results:
    print("The title is '{0}'.".format(result['title']))
    print(result)


Saw 10 result(s).
The title is '['CLIMATE ACTION - Preparation underway ahead of UN Climate Conference (COP 21) in Paris:- Opening remarks by Miguel ARIAS CAÑETE, Member of the EC in charge of Climate Action and Energy', 'CLIMATE ACTION - Preparation underway ahead of UN Climate Conference (COP 21) in Paris:- Opening remarks by Miguel ARIAS CAÑETE, Member of the EC in charge of Climate Action and Energy']'.
{'id': '/2031901/I_107857_INT_1', 'title': ['CLIMATE ACTION - Preparation underway ahead of UN Climate Conference (COP 21) in Paris:- Opening remarks by Miguel ARIAS CAÑETE, Member of the EC in charge of Climate Action and Energy', 'CLIMATE ACTION - Preparation underway ahead of UN Climate Conference (COP 21) in Paris:- Opening remarks by Miguel ARIAS CAÑETE, Member of the EC in charge of Climate Action and Energy'], 'thumbnail': ['https://api.europeana.eu/thumbnail/v2/url.json?uri=http%3A%2F%2Fdefiris.ec.streamcloud.be%2Ffindmedia%2F17%2F107857%2FTHUMB_M_I107857INT1W.jpg&type=VIDE